## 분석 환경 설정


In [ ]:
# colab tf 2.x 기본 제공하지 않음
# kearstuner 관련 소스 설치

%tensorflow_version 2.x
!pip install git+https://github.com/keras-team/keras-tuner.git

In [2]:
import os
from glob import glob
import numpy as np
import tqdm
from PIL import Image
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import utils
from tensorflow.keras import layers
from tensorflow.keras import datasets
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from kerastuner.tuners import RandomSearch
from kerastuner import HyperModel

%matplotlib inline

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## Data Loading

In [4]:
data = np.load("/content/drive/My Drive/gmmpg/zucchini_dataset.npz")
X_train, y_train, X_test, y_test = [data[f] for f in data.files]

In [5]:
# 픽셀 값 조정

X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

X_train = X_train / 255.0
X_test = X_test / 255.0

In [6]:
# one-hot encoding

y_train = utils.to_categorical(y_train, 4)
y_test = utils.to_categorical(y_test, 4)

In [7]:
class CNNHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        model = keras.Sequential()
        model.add(Conv2D(filters = hp.Choice('num_filters_1', values=[32,64], default = 32),
                         input_shape = X_train.shape[1:], kernel_size = (3, 3), padding = 'same', activation = 'relu'))
        model.add(MaxPooling2D(pool_size = (2, 2)))
        model.add(Dropout(rate=hp.Float('dropout_1', min_value=0.0, max_value=0.5,
                                        default=0.25, step=0.05)))
                                        
        
        model.add(Conv2D(filters=hp.Choice('num_filters_2', values=[32, 64], default=64),
                         activation='relu', kernel_size=3))
        model.add(MaxPooling2D(pool_size = (2, 2)))
        model.add(Dropout(rate=hp.Float('dropout_2', min_value=0.0, max_value=0.5,
                                        default=0.25, step=0.05)))
        
        model.add(Flatten())
        model.add(Dense(units=hp.Int('units', min_value=32, max_value=512, step=32, default=64),
                        activation=hp.Choice('dense_activation', values=['relu', 'tanh', 'sigmoid'], default='relu')))
        model.add(Dropout(rate=hp.Float('dropout_3', min_value=0.0, max_value=0.5,
                                        default=0.25, step=0.05)))
        
        model.add(Dense(self.num_classes, activation='softmax'))
        model.compile(optimizer=keras.optimizers.Adam(hp.Float('learning_rate',min_value=1e-4, max_value=1e-2,
                                                               sampling='LOG', default=1e-3)),
                      loss='categorical_crossentropy', metrics=['accuracy'])
        return model

In [8]:
SEED = 1
MAX_TRIALS = 15
EXECUTION_PER_TRIAL = 2

hypermodel = CNNHyperModel(input_shape=(256, 256, 3), num_classes=4)
tuner = RandomSearch(
    hypermodel,
    objective="val_accuracy",
    seed=SEED,
    max_trials=MAX_TRIALS,
    executions_per_trial=EXECUTION_PER_TRIAL,
    directory="/content/gmmpg",
    project_name="gmmpg"   
)
tuner.search_space_summary()

Search space summary
Default search space size: 8
num_filters_1 (Choice)
{'default': 32, 'conditions': [], 'values': [32, 64], 'ordered': True}
dropout_1 (Float)
{'default': 0.25, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.05, 'sampling': None}
num_filters_2 (Choice)
{'default': 64, 'conditions': [], 'values': [32, 64], 'ordered': True}
dropout_2 (Float)
{'default': 0.25, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.05, 'sampling': None}
units (Int)
{'default': 64, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
dense_activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid'], 'ordered': False}
dropout_3 (Float)
{'default': 0.25, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.05, 'sampling': None}
learning_rate (Float)
{'default': 0.001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [9]:
X_train.shape

(13388, 256, 256, 3)

In [10]:
tuner.search(X_train, y_train, batch_size=200, epochs=70, validation_split=0.2)

Trial 15 Complete [00h 27m 22s]
val_accuracy: 0.30582523345947266

Best val_accuracy So Far: 0.9536967873573303
Total elapsed time: 08h 11m 52s
INFO:tensorflow:Oracle triggered exit


In [11]:
tuner.results_summary()

Results summary
Results in /content/gmmpg/gmmpg
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
num_filters_1: 64
dropout_1: 0.35000000000000003
num_filters_2: 64
dropout_2: 0.05
units: 288
dense_activation: sigmoid
dropout_3: 0.0
learning_rate: 0.0001428379365495633
Score: 0.9536967873573303
Trial summary
Hyperparameters:
num_filters_1: 64
dropout_1: 0.05
num_filters_2: 32
dropout_2: 0.35000000000000003
units: 352
dense_activation: relu
dropout_3: 0.15000000000000002
learning_rate: 0.00014023399602325874
Score: 0.9503360986709595
Trial summary
Hyperparameters:
num_filters_1: 64
dropout_1: 0.30000000000000004
num_filters_2: 32
dropout_2: 0.2
units: 160
dense_activation: relu
dropout_3: 0.2
learning_rate: 0.0012482904754698163
Score: 0.9497759342193604
Trial summary
Hyperparameters:
num_filters_1: 64
dropout_1: 0.30000000000000004
num_filters_2: 64
dropout_2: 0.25
units: 160
dense_activation: relu
dropout_3: 0.5
learning_rate: 0.0005

In [12]:
best_models = tuner.get_best_models(num_models=1)[0]

In [13]:
loss, accuracy = best_models.evaluate(X_test, y_test)

105/105 [==============================] - 2s 18ms/step - loss: 0.1454 - accuracy: 0.9555


In [15]:
best_models.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 126, 126, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 63, 63, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 254016)            0

In [16]:
best_models.save('best_model.h5')